In [1]:
!pip install evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; pla

In [9]:
#!/usr/bin/env python3
"""
Скрипт для файнтюнинга модели Whisper на русскоязычном датасете Golos
с расчетом метрик до и после обучения.
"""

import os
import torch
import torchaudio
import numpy as np
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import json
import logging
from pathlib import Path
import evaluate
from datasets import load_dataset, DatasetDict, Audio
import jiwer
from collections import Counter
import re
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    TrainerCallback,
)
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Проверка доступности GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Используется устройство: {device}")

@dataclass
class ModelArguments:
    model_name_or_path: str = "openai/whisper-small"
    cache_dir: str = "./cache"
    use_fast_tokenizer: bool = True
    model_revision: str = "main"
    use_auth_token: bool = False

@dataclass
class DataArguments:
    dataset_name: str = "bond005/sberdevices_golos_100h_farfield"
    dataset_config_name: str = None
    train_split_name: str = "train"
    eval_split_name: str = "validation"
    audio_column_name: str = "audio"
    text_column_name: str = "sentence"  # Обновлено для нового датасета
    max_train_samples: int = None
    max_eval_samples: int = None
    max_duration_in_seconds: float = 30.0

class DataCollatorSpeechSeq2SeqWithPadding:
    """Коллатор данных для обучения Whisper"""
    
    def __init__(self, processor, decoder_start_token_id):
        self.processor = processor
        self.decoder_start_token_id = decoder_start_token_id

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Разделяем входные данные на аудио и текст
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # Создаем батч для входных аудио данных
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Создаем батч для лейблов
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Заменяем паддинг токены на -100 для игнорирования в loss
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # Если bos токен добавлен в начале, удаляем его так как он будет добавлен позже
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

def load_golos_dataset():
    """Загрузка датасета Golos от bond005"""
    try:
        # Загружаем датасет bond005/sberdevices_golos_100h_farfield
        logger.info("Загружаем датасет bond005/sberdevices_golos_100h_farfield...")
        
        # Загружаем train и validation splits
        dataset_dict = {}
        
        # Пробуем загрузить training split
        try:
            train_dataset = load_dataset(
                "bond005/sberdevices_golos_100h_farfield",
                split="train",
                streaming=False
            )
            logger.info(f"Training split загружен: {len(train_dataset)} примеров")
            dataset_dict["train"] = train_dataset
        except Exception as e:
            logger.warning(f"Не удалось загрузить training split: {e}")
        
        # Пробуем загрузить validation split
        try:
            val_dataset = load_dataset(
                "bond005/sberdevices_golos_100h_farfield",
                split="validation",
                streaming=False
            )
            logger.info(f"Validation split загружен: {len(val_dataset)} примеров")
            dataset_dict["validation"] = val_dataset
        except Exception as e:
            logger.warning(f"Не удалось загрузить validation split: {e}")
            
            # Если нет validation split, создаем его из train
            if "train" in dataset_dict:
                logger.info("Создаем validation split из training данных...")
                train_val_split = dataset_dict["train"].train_test_split(test_size=0.1, seed=42)
                dataset_dict["train"] = train_val_split["train"]
                dataset_dict["validation"] = train_val_split["test"]
        
        # Если нет ни одного split, пробуем загрузить без указания split
        if not dataset_dict:
            logger.info("Пробуем загрузить датасет без указания split...")
            full_dataset = load_dataset("bond005/sberdevices_golos_100h_farfield")
            
            # Проверяем доступные splits
            logger.info(f"Доступные splits: {list(full_dataset.keys())}")
            
            # Используем первый доступный split как train
            if full_dataset:
                first_split = list(full_dataset.keys())[0]
                logger.info(f"Используем split '{first_split}' как основной")
                
                # Создаем train/validation split
                train_val_split = full_dataset[first_split].train_test_split(test_size=0.1, seed=42)
                dataset_dict["train"] = train_val_split["train"]
                dataset_dict["validation"] = train_val_split["test"]
        
        if not dataset_dict:
            raise ValueError("Не удалось загрузить ни один split датасета")
        
        # Проверяем структуру данных
        sample = dataset_dict["train"][0]
        logger.info(f"Пример структуры данных: {list(sample.keys())}")
        
        # Проверяем наличие нужных колонок
        required_columns = ["audio", "sentence"]
        available_columns = list(sample.keys())
        
        for col in required_columns:
            if col not in available_columns:
                logger.warning(f"Колонка '{col}' не найдена. Доступные колонки: {available_columns}")
                
                # Пробуем найти альтернативные названия
                if col == "sentence":
                    alternatives = ["text", "transcription", "transcript", "target_text"]
                    for alt in alternatives:
                        if alt in available_columns:
                            logger.info(f"Используем колонку '{alt}' вместо '{col}'")
                            break
                    else:
                        logger.error(f"Не найдена подходящая текстовая колонка")
        
        logger.info("Датасет Golos загружен успешно")
        return DatasetDict(dataset_dict)
        
    except Exception as e:
        logger.error(f"Ошибка загрузки датасета Golos: {e}")
        logger.info("Создается демонстрационный датасет...")
        return create_dummy_dataset()

def create_dummy_dataset():
    """Создание реалистичного демонстрационного датасета для тестирования"""
    logger.warning("Создается демонстрационный датасет с синтезированной русской речью")
    
    from datasets import Dataset
    
    # Создаем более реалистичные аудио данные
    sample_rate = 16000
    
    # Русские фразы для синтеза
    russian_phrases = [
        "Привет как дела",
        "Сегодня хорошая погода",
        "Я изучаю машинное обучение",
        "Whisper работает с русским языком",
        "Нейронные сети очень интересны",
        "Москва столица России",
        "Искусственный интеллект развивается быстро",
        "Давайте попробуем распознать речь",
        "Это тестовый пример для обучения",
        "Русский язык имеет сложную грамматику",
        "Автоматическое распознавание речи",
        "Файнтюнинг модели на русских данных",
        "Качество распознавания улучшается",
        "Глубокое обучение показывает хорошие результаты",
        "Обработка естественного языка"
    ]
    
    dummy_data = []
    for i in range(200):  # Больше примеров для лучшего обучения
        phrase_idx = i % len(russian_phrases)
        phrase = russian_phrases[phrase_idx]
        
        # Создаем более реалистичный аудио сигнал
        # Имитируем речевой сигнал с основной частотой и формантами
        duration = len(phrase.split()) * 0.4 + np.random.uniform(0.5, 1.0)  # Реалистичная длительность
        num_samples = int(duration * sample_rate)
        
        # Создаем базовый сигнал с речевыми характеристиками
        t = np.linspace(0, duration, num_samples)
        fundamental_freq = np.random.uniform(80, 200)  # Основная частота голоса
        
        # Имитируем речевой сигнал
        signal = np.zeros(num_samples)
        for harmonic in range(1, 6):
            amplitude = 1.0 / harmonic
            signal += amplitude * np.sin(2 * np.pi * fundamental_freq * harmonic * t)
        
        # Добавляем формантные частоты
        for formant_freq in [500, 1500, 2500]:
            formant_signal = 0.3 * np.sin(2 * np.pi * formant_freq * t)
            signal += formant_signal
        
        # Добавляем огибающую и шум
        envelope = np.exp(-t * 0.5)  # Экспоненциальная огибающая
        noise = 0.1 * np.random.randn(num_samples)
        signal = signal * envelope + noise
        
        # Нормализация
        signal = signal / np.max(np.abs(signal)) * 0.7
        audio_array = signal.astype(np.float32)
        
        dummy_data.append({
            "audio": {"array": audio_array, "sampling_rate": sample_rate},
            "sentence": phrase  # Изменено с "transcription" на "sentence"
        })
    
    dataset = Dataset.from_list(dummy_data)
    
    # Создаем train/validation split
    dataset = dataset.train_test_split(test_size=0.15, seed=42)
    
    return DatasetDict({
        "train": dataset["train"],
        "validation": dataset["test"]
    })

def prepare_dataset(batch, processor, normalizer, text_column_name="sentence"):
    """Предобработка данных"""
    # Загружаем аудио
    audio = batch["audio"]
    
    # Вычисляем log-Mel спектрограммы
    input_features = processor.feature_extractor(
        audio["array"], 
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    
    # Нормализация и токенизация текста
    transcription = batch[text_column_name]
    if normalizer:
        transcription = normalizer(transcription)
    
    # Кодируем текст
    labels = processor.tokenizer(transcription).input_ids
    
    return {
        "input_features": input_features,
        "labels": labels
    }

def compute_metrics(eval_preds, processor, normalizer, metrics_dict):
    """Вычисление расширенного набора метрик для ASR"""
    pred_ids, label_ids = eval_preds
    
    # Заменяем -100 на pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    
    # Декодируем предсказания и истинные лейблы
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    # Нормализация для более справедливого сравнения
    if normalizer:
        pred_str = [normalizer(pred) for pred in pred_str]
        label_str = [normalizer(label) for label in label_str]
    
    # Основные метрики
    results = {}
    
    # 1. WER (Word Error Rate)
    wer = 100 * metrics_dict["wer"].compute(predictions=pred_str, references=label_str)
    results["wer"] = wer
    
    # 2. CER (Character Error Rate)
    cer = 100 * metrics_dict["cer"].compute(predictions=pred_str, references=label_str)
    results["cer"] = cer
    
    # 3. BLEU Score (для оценки качества текста)
    try:
        bleu = metrics_dict["bleu"].compute(predictions=pred_str, references=[[ref] for ref in label_str])
        results["bleu"] = bleu["bleu"] * 100
    except:
        results["bleu"] = 0.0
    
    # 4. Детальные метрики с использованием jiwer
    try:
        # Объединяем все предсказания и референсы для общей статистики
        all_preds = " ".join(pred_str)
        all_refs = " ".join(label_str)
        
        # Подсчет операций редактирования
        measures = jiwer.compute_measures(all_refs, all_preds)
        
        results["substitutions"] = measures["substitutions"]
        results["deletions"] = measures["deletions"] 
        results["insertions"] = measures["insertions"]
        results["hits"] = measures["hits"]
        
        # Дополнительные метрики
        total_words = measures["substitutions"] + measures["deletions"] + measures["hits"]
        if total_words > 0:
            results["substitution_rate"] = (measures["substitutions"] / total_words) * 100
            results["deletion_rate"] = (measures["deletions"] / total_words) * 100
            results["insertion_rate"] = (measures["insertions"] / (total_words + measures["insertions"])) * 100
        else:
            results["substitution_rate"] = 0.0
            results["deletion_rate"] = 0.0
            results["insertion_rate"] = 0.0
            
    except Exception as e:
        logger.warning(f"Ошибка при вычислении детальных метрик: {e}")
        results.update({
            "substitutions": 0, "deletions": 0, "insertions": 0, "hits": 0,
            "substitution_rate": 0.0, "deletion_rate": 0.0, "insertion_rate": 0.0
        })
    
    # 5. Длина предсказаний (для анализа)
    avg_pred_length = sum(len(pred.split()) for pred in pred_str) / len(pred_str)
    avg_ref_length = sum(len(ref.split()) for ref in label_str) / len(label_str)
    results["avg_prediction_length"] = avg_pred_length
    results["avg_reference_length"] = avg_ref_length
    results["length_ratio"] = avg_pred_length / avg_ref_length if avg_ref_length > 0 else 0.0
    
    # 6. Точность на уровне предложений (Sentence Accuracy)
    exact_matches = sum(1 for pred, ref in zip(pred_str, label_str) if pred.strip() == ref.strip())
    sentence_accuracy = (exact_matches / len(pred_str)) * 100
    results["sentence_accuracy"] = sentence_accuracy
    
    return results

def evaluate_model(model, processor, eval_dataset, normalizer, metrics_dict, max_samples=50):
    """Оценка модели на валидационном наборе с расширенными метриками"""
    model.eval()
    
    predictions = []
    references = []
    
    # Ограничиваем количество примеров для быстрой оценки
    eval_subset = eval_dataset.select(range(min(max_samples, len(eval_dataset))))
    
    with torch.no_grad():
        for i, batch in enumerate(eval_subset):
            if i % 10 == 0:
                logger.info(f"Обработано {i}/{len(eval_subset)} примеров")
            
            # Подготавливаем входные данные
            input_features = torch.tensor(batch["input_features"]).unsqueeze(0).to(device)
            
            # Генерируем предсказание
            predicted_ids = model.generate(
                input_features,
                max_length=225,
                num_beams=1,
                do_sample=False
            )
            
            # Декодируем
            pred_text = processor.tokenizer.decode(predicted_ids[0], skip_special_tokens=True)
            true_text = processor.tokenizer.decode(batch["labels"], skip_special_tokens=True)
            
            # Нормализация
            if normalizer:
                pred_text = normalizer(pred_text)
                true_text = normalizer(true_text)
            
            predictions.append(pred_text)
            references.append(true_text)
    
    # Вычисляем все метрики
    results = {}
    
    # WER
    wer = 100 * metrics_dict["wer"].compute(predictions=predictions, references=references)
    results["wer"] = wer
    
    # CER
    cer = 100 * metrics_dict["cer"].compute(predictions=predictions, references=references)
    results["cer"] = cer
    
    # BLEU
    try:
        bleu = metrics_dict["bleu"].compute(predictions=predictions, references=[[ref] for ref in references])
        results["bleu"] = bleu["bleu"] * 100
    except:
        results["bleu"] = 0.0
    
    # Детальная статистика ошибок
    try:
        all_preds = " ".join(predictions)
        all_refs = " ".join(references)
        measures = jiwer.compute_measures(all_refs, all_preds)
        
        results["substitutions"] = measures["substitutions"]
        results["deletions"] = measures["deletions"]
        results["insertions"] = measures["insertions"]
        results["hits"] = measures["hits"]
        
        # Rates
        total_words = measures["substitutions"] + measures["deletions"] + measures["hits"]
        if total_words > 0:
            results["substitution_rate"] = (measures["substitutions"] / total_words) * 100
            results["deletion_rate"] = (measures["deletions"] / total_words) * 100
            results["insertion_rate"] = (measures["insertions"] / (total_words + measures["insertions"])) * 100
        
    except Exception as e:
        logger.warning(f"Ошибка при вычислении детальных метрик: {e}")
    
    # Sentence Accuracy
    exact_matches = sum(1 for pred, ref in zip(predictions, references) if pred.strip() == ref.strip())
    results["sentence_accuracy"] = (exact_matches / len(predictions)) * 100
    
    # Длина предсказаний
    avg_pred_length = sum(len(pred.split()) for pred in predictions) / len(predictions)
    avg_ref_length = sum(len(ref.split()) for ref in references) / len(references)
    results["avg_prediction_length"] = avg_pred_length
    results["avg_reference_length"] = avg_ref_length
    results["length_ratio"] = avg_pred_length / avg_ref_length if avg_ref_length > 0 else 0.0
    
    # Выводим примеры и метрики
    logger.info("\n=== Примеры предсказаний ===")
    for i in range(min(5, len(predictions))):
        logger.info(f"Истинный текст: {references[i]}")
        logger.info(f"Предсказание:   {predictions[i]}")
        logger.info("---")
    
    logger.info("\n=== Детальные метрики ===")
    logger.info(f"WER: {results['wer']:.2f}%")
    logger.info(f"CER: {results['cer']:.2f}%")
    logger.info(f"BLEU: {results['bleu']:.2f}")
    logger.info(f"Sentence Accuracy: {results['sentence_accuracy']:.2f}%")
    
    if "substitutions" in results:
        logger.info(f"Substitutions: {results['substitutions']}")
        logger.info(f"Deletions: {results['deletions']}")
        logger.info(f"Insertions: {results['insertions']}")
        logger.info(f"Hits: {results['hits']}")
        logger.info(f"Substitution Rate: {results.get('substitution_rate', 0):.2f}%")
        logger.info(f"Deletion Rate: {results.get('deletion_rate', 0):.2f}%")
        logger.info(f"Insertion Rate: {results.get('insertion_rate', 0):.2f}%")
    
    logger.info(f"Avg Prediction Length: {results['avg_prediction_length']:.1f} words")
    logger.info(f"Avg Reference Length: {results['avg_reference_length']:.1f} words")
    logger.info(f"Length Ratio: {results['length_ratio']:.2f}")
    
    results["predictions"] = predictions
    results["references"] = references
    
    return results

class MetricsCallback(TrainerCallback):
    """Коллбэк для логирования расширенных метрик"""
    
    def on_evaluate(self, args, state, control, model, logs=None, **kwargs):
        if logs:
            logger.info(f"\nЭпоха {state.epoch} - Метрики оценки:")
            logger.info(f"  WER: {logs.get('eval_wer', 'N/A'):.2f}%")
            logger.info(f"  CER: {logs.get('eval_cer', 'N/A'):.2f}%")
            logger.info(f"  BLEU: {logs.get('eval_bleu', 'N/A'):.2f}")
            logger.info(f"  Sentence Accuracy: {logs.get('eval_sentence_accuracy', 'N/A'):.2f}%")
            
            if 'eval_substitutions' in logs:
                logger.info(f"  Substitutions: {logs.get('eval_substitutions', 0)}")
                logger.info(f"  Deletions: {logs.get('eval_deletions', 0)}")
                logger.info(f"  Insertions: {logs.get('eval_insertions', 0)}")
                logger.info(f"  Substitution Rate: {logs.get('eval_substitution_rate', 0):.2f}%")

def main():
    """Основная функция файнтюнинга"""
    
    # Аргументы модели и данных
    model_args = ModelArguments()
    data_args = DataArguments()
    
    # Загружаем процессор и модель
    logger.info("Загрузка модели и процессора...")
    
    # Важно: правильно настраиваем язык и задачу
    feature_extractor = WhisperFeatureExtractor.from_pretrained(model_args.model_name_or_path)
    tokenizer = WhisperTokenizer.from_pretrained(
        model_args.model_name_or_path, 
        language="ru",  # Используем короткий код языка
        task="transcribe"
    )
    processor = WhisperProcessor.from_pretrained(
        model_args.model_name_or_path, 
        language="ru",  # Используем короткий код языка
        task="transcribe"
    )
    
    model = WhisperForConditionalGeneration.from_pretrained(model_args.model_name_or_path)
    model.to(device)
    
    # КРИТИЧЕСКИ ВАЖНО: правильная настройка модели для русского языка
    model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(
        language="ru", 
        task="transcribe"
    )
    
    # НЕ подавляем все токены - это может нарушить работу модели
    # model.config.suppress_tokens = []
    
    logger.info(f"Модель настроена для русского языка")
    logger.info(f"Forced decoder IDs: {model.config.forced_decoder_ids}")
    
    # Загружаем датасет
    logger.info("Загрузка датасета...")
    raw_datasets = load_golos_dataset()
    
    # Определяем название текстовой колонки
    sample = raw_datasets["train"][0]
    text_column_name = data_args.text_column_name
    
    # Проверяем, есть ли нужная колонка
    if text_column_name not in sample:
        # Ищем альтернативные названия
        alternatives = ["text", "transcription", "transcript", "target_text", "sentence"]
        for alt in alternatives:
            if alt in sample:
                text_column_name = alt
                logger.info(f"Используем колонку '{text_column_name}' для текста")
                break
        else:
            logger.error(f"Не найдена текстовая колонка. Доступные: {list(sample.keys())}")
            return
    
    # Ресэмплируем аудио если необходимо
    raw_datasets = raw_datasets.cast_column(
        data_args.audio_column_name, 
        Audio(sampling_rate=feature_extractor.sampling_rate)
    )
    
    # Нормализатор текста
    normalizer = BasicTextNormalizer()
    
    # Предобработка данных
    logger.info("Предобработка данных...")
    vectorized_datasets = raw_datasets.map(
        lambda batch: prepare_dataset(batch, processor, normalizer, text_column_name),
        remove_columns=raw_datasets["train"].column_names,
        desc="Предобработка данных"
    )
    
    # Ограничиваем размер датасета если указано
    if data_args.max_train_samples:
        vectorized_datasets["train"] = vectorized_datasets["train"].select(
            range(min(data_args.max_train_samples, len(vectorized_datasets["train"])))
        )
    
    if data_args.max_eval_samples:
        vectorized_datasets["validation"] = vectorized_datasets["validation"].select(
            range(min(data_args.max_eval_samples, len(vectorized_datasets["validation"])))
        )
    
    # Коллатор данных
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(
        processor=processor,
        decoder_start_token_id=model.config.decoder_start_token_id,
    )
    
    # Загружаем все метрики
    logger.info("Инициализация метрик...")
    metrics_dict = {
        "wer": evaluate.load("wer"),
        "cer": evaluate.load("cer"),
        "bleu": evaluate.load("bleu")
    }
    
    logger.info("Доступные метрики: WER, CER, BLEU, Sentence Accuracy, подробная статистика ошибок")
    
    # Оценка до файнтюнинга
    logger.info("\n" + "="*50)
    logger.info("ОЦЕНКА МОДЕЛИ ДО ФАЙНТЮНИНГА")
    logger.info("="*50)
    
    pre_finetune_results = evaluate_model(
        model, processor, vectorized_datasets["validation"], 
        normalizer, metrics_dict, max_samples=20
    )
    logger.info(f"WER до файнтюнинга: {pre_finetune_results['wer']:.2f}%")
    
    # Функция для вычисления метрик
    def compute_metrics_wrapper(eval_preds):
        return compute_metrics(eval_preds, processor, normalizer, metrics_dict)
    
    # Аргументы обучения
    training_args = Seq2SeqTrainingArguments(
        output_dir="./whisper-golos-finetuned",
        per_device_train_batch_size=4,  # Уменьшаем batch size
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,  # Увеличиваем накопление градиентов
        learning_rate=5e-6,  # Более консервативный learning rate
        warmup_steps=100,
        max_steps=1000,  # Меньше шагов для демонстрации
        gradient_checkpointing=True,
        fp16=True if device == "cuda" else False,
        eval_steps=200,
        save_strategy="steps",
        save_steps=200,
        logging_steps=50,
        report_to=["tensorboard"],
        metric_for_best_model="wer",
        greater_is_better=False,
        push_to_hub=False,
        dataloader_num_workers=0,
        predict_with_generate=True,
        generation_max_length=225,
        generation_num_beams=2,  # Beam search для лучшего качества
        save_total_limit=2,
        # Добавляем параметры для стабильности обучения
        dataloader_drop_last=True,
        remove_unused_columns=False,
    )
    
    # Создаем тренер
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=vectorized_datasets["train"],
        eval_dataset=vectorized_datasets["validation"],
        tokenizer=processor.feature_extractor,
        data_collator=data_collator,
        compute_metrics=compute_metrics_wrapper,
        callbacks=[MetricsCallback()],
    )
    
    # Обучение
    logger.info("\n" + "="*50)
    logger.info("НАЧАЛО ФАЙНТЮНИНГА")
    logger.info("="*50)
    
    try:
        train_result = trainer.train()
        logger.info(f"Обучение завершено за {train_result.metrics['train_runtime']:.2f} секунд")
    except Exception as e:
        logger.error(f"Ошибка во время обучения: {e}")
        return
    
    # Оценка после файнтюнинга
    logger.info("\n" + "="*50)
    logger.info("ОЦЕНКА МОДЕЛИ ПОСЛЕ ФАЙНТЮНИНГА")
    logger.info("="*50)
    
    post_finetune_results = evaluate_model(
        model, processor, vectorized_datasets["validation"], 
        normalizer, metrics_dict, max_samples=20
    )
    logger.info(f"WER после файнтюнинга: {post_finetune_results['wer']:.2f}%")
    
    # Сравнение результатов
    wer_improvement = pre_finetune_results['wer'] - post_finetune_results['wer']
    cer_improvement = pre_finetune_results['cer'] - post_finetune_results['cer']
    bleu_improvement = post_finetune_results['bleu'] - pre_finetune_results['bleu']
    accuracy_improvement = post_finetune_results['sentence_accuracy'] - pre_finetune_results['sentence_accuracy']
    
    logger.info("\n" + "="*60)
    logger.info("ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
    logger.info("="*60)
    logger.info("ОСНОВНЫЕ МЕТРИКИ:")
    logger.info(f"  WER до файнтюнинга:    {pre_finetune_results['wer']:.2f}%")
    logger.info(f"  WER после файнтюнинга: {post_finetune_results['wer']:.2f}%")
    logger.info(f"  Улучшение WER:         {wer_improvement:.2f}% {'(лучше)' if wer_improvement > 0 else '(хуже)'}")
    logger.info("")
    logger.info(f"  CER до файнтюнинга:    {pre_finetune_results['cer']:.2f}%")
    logger.info(f"  CER после файнтюнинга: {post_finetune_results['cer']:.2f}%")
    logger.info(f"  Улучшение CER:         {cer_improvement:.2f}% {'(лучше)' if cer_improvement > 0 else '(хуже)'}")
    logger.info("")
    logger.info(f"  BLEU до файнтюнинга:    {pre_finetune_results['bleu']:.2f}")
    logger.info(f"  BLEU после файнтюнинга: {post_finetune_results['bleu']:.2f}")
    logger.info(f"  Улучшение BLEU:         {bleu_improvement:.2f} {'(лучше)' if bleu_improvement > 0 else '(хуже)'}")
    logger.info("")
    logger.info(f"  Sentence Accuracy до:    {pre_finetune_results['sentence_accuracy']:.2f}%")
    logger.info(f"  Sentence Accuracy после: {post_finetune_results['sentence_accuracy']:.2f}%")
    logger.info(f"  Улучшение Accuracy:      {accuracy_improvement:.2f}% {'(лучше)' if accuracy_improvement > 0 else '(хуже)'}")
    
    # Детальная статистика ошибок
    if 'substitutions' in post_finetune_results:
        logger.info("\nДЕТАЛЬНАЯ СТАТИСТИКА ОШИБОК (после файнтюнинга):")
        logger.info(f"  Правильно распознано слов: {post_finetune_results['hits']}")
        logger.info(f"  Замены (substitutions):    {post_finetune_results['substitutions']}")
        logger.info(f"  Удаления (deletions):      {post_finetune_results['deletions']}")
        logger.info(f"  Вставки (insertions):      {post_finetune_results['insertions']}")
        logger.info(f"  Частота замен:             {post_finetune_results.get('substitution_rate', 0):.2f}%")
        logger.info(f"  Частота удалений:          {post_finetune_results.get('deletion_rate', 0):.2f}%")
        logger.info(f"  Частота вставок:           {post_finetune_results.get('insertion_rate', 0):.2f}%")
    
    logger.info("\nАНАЛИЗ ДЛИНЫ ПРЕДСКАЗАНИЙ:")
    logger.info(f"  Средняя длина референса:    {post_finetune_results['avg_reference_length']:.1f} слов")
    logger.info(f"  Средняя длина предсказания: {post_finetune_results['avg_prediction_length']:.1f} слов")
    logger.info(f"  Соотношение длин:           {post_finetune_results['length_ratio']:.2f}")
    logger.info("="*60)
    
    # Сохранение модели
    logger.info("\nСохранение модели...")
    model_save_path = "./whisper_golos.pt"
    
    # Сохраняем состояние модели
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': model.config,
        'metrics_before': {
            'wer': pre_finetune_results['wer'],
            'cer': pre_finetune_results['cer'],
            'bleu': pre_finetune_results['bleu'],
            'sentence_accuracy': pre_finetune_results['sentence_accuracy']
        },
        'metrics_after': {
            'wer': post_finetune_results['wer'],
            'cer': post_finetune_results['cer'],
            'bleu': post_finetune_results['bleu'],
            'sentence_accuracy': post_finetune_results['sentence_accuracy']
        },
        'improvements': {
            'wer': wer_improvement,
            'cer': cer_improvement,
            'bleu': bleu_improvement,
            'sentence_accuracy': accuracy_improvement
        }
    }, model_save_path)
    
    # Также сохраняем полную модель в формате HuggingFace
    model.save_pretrained("./whisper-golos-final")
    processor.save_pretrained("./whisper-golos-final")
    
    logger.info(f"Модель сохранена в {model_save_path}")
    logger.info("Полная модель сохранена в ./whisper-golos-final/")
    
    # Сохраняем детальные результаты
    results = {
        "model_name": model_args.model_name_or_path,
        "dataset": data_args.dataset_name,
        "metrics_before_finetune": {
            "wer": pre_finetune_results['wer'],
            "cer": pre_finetune_results['cer'],
            "bleu": pre_finetune_results['bleu'],
            "sentence_accuracy": pre_finetune_results['sentence_accuracy'],
            "avg_prediction_length": pre_finetune_results['avg_prediction_length'],
            "avg_reference_length": pre_finetune_results['avg_reference_length'],
            "length_ratio": pre_finetune_results['length_ratio']
        },
        "metrics_after_finetune": {
            "wer": post_finetune_results['wer'],
            "cer": post_finetune_results['cer'],
            "bleu": post_finetune_results['bleu'],
            "sentence_accuracy": post_finetune_results['sentence_accuracy'],
            "avg_prediction_length": post_finetune_results['avg_prediction_length'],
            "avg_reference_length": post_finetune_results['avg_reference_length'],
            "length_ratio": post_finetune_results['length_ratio']
        },
        "improvements": {
            "wer": wer_improvement,
            "cer": cer_improvement,
            "bleu": bleu_improvement,
            "sentence_accuracy": accuracy_improvement
        },
        "training_steps": training_args.max_steps,
        "examples": {
            "references": post_finetune_results['references'][:5],
            "predictions": post_finetune_results['predictions'][:5]
        }
    }
    
    # Добавляем детальную статистику ошибок если доступна
    if 'substitutions' in post_finetune_results:
        results["error_analysis"] = {
            "hits": post_finetune_results['hits'],
            "substitutions": post_finetune_results['substitutions'],
            "deletions": post_finetune_results['deletions'],
            "insertions": post_finetune_results['insertions'],
            "substitution_rate": post_finetune_results.get('substitution_rate', 0),
            "deletion_rate": post_finetune_results.get('deletion_rate', 0),
            "insertion_rate": post_finetune_results.get('insertion_rate', 0)
        }
        
        results["examples"] = {
            "references": post_finetune_results['references'][:5],
            "predictions": post_finetune_results['predictions'][:5]
        }
    
    with open("finetune_results.json", "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    logger.info("Детальные результаты сохранены в finetune_results.json")
    logger.info("Файнтюнинг завершен успешно!")

if __name__ == "__main__":
    main()

/tmp/ipykernel_35/3084336095.py:645: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss
50,1.011000
100,0.409500
150,0.281900
200,0.248400
250,0.273000
300,0.242300
350,0.255500
400,0.235000
450,0.234400
500,0.220900


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [10]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

In [12]:
download_file('/kaggle/working/whisper-golos-final', 'whisper_final')

/kaggle/working/whisper_final.zip